# Start here if this is your first run today 🚀

**Import the necessary raw data to obtain the most recent updates.**

In [1]:
# Library list🤖
import glob, warnings, polars as pl, datetime, sqlite3, time, os, zipfile, xml.dom.minidom
from datetime import datetime as dt, time as t, timedelta
import pandas as pd, numpy as np, sqlalchemy as sa, xlsxwriter
from sqlalchemy import create_engine
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from polars.exceptions import ColumnNotFoundError
import shutil

# -----------------------------------------------------------------------------------------------#
# Source collection
user_credential = os.path.join(os.environ['USERPROFILE'],r'Concentrix Corporation//CNXVN - WFM Team - Documents//')

# INPUT-----💾-----💾-----💾-----💾-----💾-----💾-----💾-----💾-----💾-----💾

# [BKN]AGENTS 0️⃣1️⃣
Link_AGENTS = os.path.join(user_credential,
                            r'DataBase//DataRaw//BKN//AGENTS//*.xlsx')
# [GLB]Ramco 0️⃣2️⃣
Link_Ramco = os.path.join(user_credential,
                            r'DataBase//DataRaw//GLOBAL//RAMCO//*.CSV')
# [BKN]Schedule 0️⃣3️⃣
Link_Schedule = os.path.join(user_credential,
                            r'DataBase//DataRaw//BKN//ROSTER//*.xlsx')
# [BKN]IEX 0️⃣4️⃣
Link_IEX = os.path.join(user_credential,
                        r'DataBase//DataRaw//BKN//IEX//*.xlsx')
# [BKN]Workplan 0️⃣5️⃣
Link_Workplan = os.path.join(user_credential,
                            r'DataBase//DataRaw//BKN//WORKPLAN//WORKPLAN QUERY//*.xlsx')
# [BKN]REALTIME_CUIC 0️⃣6️⃣
Link_REALTIME_CUIC = os.path.join(user_credential,
                            r'DataBase//DataRaw//BKN//REALTIME_VIEW//*.xlsx')

# OUTPUT-----📥-----📥-----📥-----📥-----📥-----📥-----📥-----📥-----📥-----📥

# [BKN]MONITOR_DF 📑
DF_MONITOR_DF = os.path.join(user_credential, 
                        r'DataBase//DataFrame//BKN//MONITOR_DF')

# -----------------------------------------------------------------------------------------------#
# MyMiscellaneous

# [DB]TonyMiscellaneous
Link_DB = os.path.join(os.environ['USERPROFILE'], r'Desktop//Bcom_DB.db')
conn = create_engine(f"sqlite:///{Link_DB}")

In [2]:
#[BKN]Raw_Agents🎯
 
Data_AGENTS = []
for file in glob.glob(Link_AGENTS):
    filename = os.path.basename(file)
    Read = pl.read_excel(file, infer_schema_length=None,schema_overrides={"serial_number": pl.Utf8},engine="calamine",sheet_name="Sheet1")
    AGENTS = Read.select(pl.all(), pl.lit(filename).alias("Filename"))
    Data_AGENTS.append(AGENTS)
AGENTS = pl.concat(Data_AGENTS, how="diagonal")

#Edit Column
# AGENTS = AGENTS.with_columns(pl.col('PST_Start_Date','Production_Start_Date','Language Start Date','Hire_Date','LWD','Termination_Date').dt.strftime('%Y-%m-%d'),)
AGENTS = AGENTS.select(['Filename','Employee_ID','GEO','Site_ID','Employee_Last_Name','Employee_First_Name',
                        'Status','Wave #','Role','Booking Login ID','Language Start Date','TED Name','CUIC Name','PST_Start_Date',
                        'Production_Start_Date','LWD','Designation','cnx_email','Booking Email','WAH Category','Full name','IEX',
                        'serial_number','BKN_ID','Extension Number'])
AGENTS = AGENTS.with_columns(pl.col('Extension Number').cast(pl.Int64, strict=False))

# Export to Database
Export_AGENTS_DB = AGENTS.write_database("AGENTS_RAW", f"sqlite:///{Link_DB}", if_table_exists="replace")

# #RUN
Export_AGENTS_DB
AGENTS.sample(n=3)

Filename,Employee_ID,GEO,Site_ID,Employee_Last_Name,Employee_First_Name,Status,Wave #,Role,Booking Login ID,Language Start Date,TED Name,CUIC Name,PST_Start_Date,Production_Start_Date,LWD,Designation,cnx_email,Booking Email,WAH Category,Full name,IEX,serial_number,BKN_ID,Extension Number
str,i64,str,str,str,str,str,str,str,str,date,str,str,date,date,date,str,str,str,str,str,i64,str,str,i64
"""CNX Global Master Roster.xlsx""",102238574,"""APAC""","""HCM4""","""Nguyen""","""Cong Thanh""","""Transferred""","""68""","""Agent""","""anguyen35""",2023-05-22,"""Aiden Nguyen""","""Nguyen, Aiden""",2023-05-08,2023-05-22,2024-12-18,"""Production""","""congthanh.nguyen1@concentrix.c…","""aiden.nguyen@booking.com""","""On Site""","""Nguyen Cong Thanh""",3064393,"""HCM_1334_ANGU""","""131311""",72786566
"""CNX Global Master Roster.xlsx""",102893357,"""APAC""","""HCM4""","""Do""","""Dinh Khanh""","""Active""","""139""","""Agent""","""ndo5""",2024-07-23,"""Neexie Do""","""Do, Neexie""",2024-07-16,2024-08-05,null,"""Agent""","""dinhkhanh.do@concentrix.com""","""neexie.do@booking.com""","""On Site""","""Do Dinh Khanh""",3021156,"""HCM_2100_NDO""","""155909""",72787230
"""CNX Global Master Roster.xlsx""",102850061,"""APAC""","""HCM4""","""LE""","""MINH SANG""","""Termed""","""118""","""Agent""","""ale14""",2024-05-13,"""Axl Le""","""Le, Axl""",2024-05-20,2024-06-10,2024-09-15,"""Production""","""minhsang.le@concentrix.com""","""axl.le@booking.com""","""On Site""","""LE MINH SANG""",3009169,"""HCM_1757_ALE""","""151033""",72786938


In [3]:
#[GLB]Ramco🎯

Data_Ramco = []
for file in glob.glob(Link_Ramco):
    filename = os.path.basename(file)
    Read = pl.read_csv(file, infer_schema_length=0, encoding='latin-1')
    Ramco = Read.select(pl.all(), pl.lit(filename).alias("Filename"))
    Data_Ramco.append(Ramco)
Ramco = pl.concat(Data_Ramco, how="diagonal")
 
#Edit Column
Ramco = Ramco.with_columns(pl.col('Attribute').str.strptime(pl.Date, format='%m/%d/%Y'),
                                         pl.col('EID').cast(pl.Int64)
            ).rename({'Attribute': 'Date'}).select(['Filename','EID','Employee_Name','Employee_type','Date','Value'])
 
# Export to Database
Export_Ramco_DB = Ramco.write_database("Ramco_RAW", f"sqlite:///{Link_DB}", if_table_exists="replace")
 
#RUN
Export_Ramco_DB
Ramco.sample(n=3)

Filename,EID,Employee_Name,Employee_type,Date,Value
str,i64,str,str,date,str
"""20230101_20231231.csv""",102900538,"""TRONG NHAN LUU""","""Fixed Term Hire""",2023-08-24,"""NM"""
"""20220901_20221231.csv""",102055498,"""DAC VU""","""Fixed Term Hire""",2022-09-03,"""PR"""
"""20220901_20221231.csv""",102154676,"""THI THU TRANG LE""","""Regular""",2022-12-07,"""PR"""


In [4]:
#[BKN]Schedule🎯

Data_Schedule = []
for file in glob.glob(Link_Schedule):
    filename = os.path.basename(file)
    Read = pl.read_excel(file, infer_schema_length=None, engine="calamine", sheet_name="Sheet1")
    Schedule = Read.select(pl.all(), pl.lit(filename).alias("Filename"))
    Data_Schedule.append(Schedule)
Schedule = pl.concat(Data_Schedule, how="diagonal")

#Edit Column
# Schedule = Schedule.with_columns(pl.col("Attribute").str.to_datetime("%m-%d-%y").cast(pl.Date)) 
Schedule = Schedule.with_columns(pl.col("Value").alias("Original_Shift"))
condition = (
    (pl.col("Value") == "OFF")
    | (pl.col("Value") == "AL")
    | (pl.col("Value") == "CO")
    | (pl.col("Value") == "HO")
    | (pl.col("Value") == "UPL")
    | (pl.col("Value") == "VGH"))
condition_training = (
    (pl.col("Value") == "Training")
    | (pl.col("Value") == "PEGA"))
Schedule = Schedule.with_columns(
    pl.when(condition).then(pl.lit("OFF"))
      .when(condition_training).then(pl.lit("Training"))
      .otherwise(pl.col("NS Check")).alias("Shift_type"))
Schedule = Schedule.rename({'Attribute':'Date','Value':'Shift'}).drop('SEAT')
Schedule = Schedule.join(Ramco.select(pl.col(["EID", "Date", "Value"])), coalesce=True,
                         left_on=["Emp ID", "Date"],right_on=["EID", "Date"],how="left")
Schedule = Schedule.with_columns(
    pl.when(
        (pl.col("Value") == "PH") | (pl.col("Value") == "PO")
    ).then(
        pl.col("week_shift")
    ).otherwise(
        pl.col("Shift")
    ).alias("Actual_shift"))
Schedule = Schedule.with_columns(
    pl.when(
        pl.col("Actual_shift").is_in([
            '0000-0900', '0100-1000', '0200-1100', '0300-1200', '0400-1300', '0500-1400',
            '0600-1500', '0700-1600', '0800-1700', '0900-1800', '1000-1900', '1100-2000',
            '1200-2100', '1300-2200', '1400-2300'])
    ).then(pl.lit("DS"))
    .when(
        pl.col("Actual_shift").is_in([
            '1500-0000', '1600-0100', '1700-0200', '1800-0300', '1900-0400', '2000-0500',
            '2100-0600', '2200-0700', '2300-0800'])
    ).then(pl.lit("NS"))
    .otherwise(pl.col("Shift_type"))
    .fill_null(pl.col("Shift_type"))  
    .alias("Actual_shift_type"))
Schedule = Schedule.select(['Filename','LOB','DPE','OM','team_leader',
                            'Emp ID','Name','Date','Actual_shift','Actual_shift_type', 
                            'week_off','Original_Shift','week_shift'])
Schedule = Schedule.rename({'Actual_shift':'Shift','Actual_shift_type':'Shift_type'})

# Export to Database
Export_Schedule_DB = Schedule.write_database("Schedule_RAW", f"sqlite:///{Link_DB}", if_table_exists="replace")

#RUN
Export_Schedule_DB
Schedule.sample(n=3)

Filename,LOB,DPE,OM,team_leader,Emp ID,Name,Date,Shift,Shift_type,week_off,Original_Shift,week_shift
str,str,i64,i64,i64,i64,str,date,str,str,str,str,str
"""20240701_20241229.xlsx""","""DE""",516333,101755109,101993760,102476823,"""Nguyen Tai Nam Anh""",2024-09-16,"""1900-0400""","""NS""","""WedThu""","""1900-0400""","""1900-0400"""
"""20240101_20240630.xlsx""","""VICSP""",101731723,101791108,102004295,102278405,"""Thong Kỳ Nam""",2024-06-18,"""OFF""","""OFF""","""TueWed""","""OFF""","""0900-1800"""
"""20240701_20241229.xlsx""","""IT""",101731723,101791108,102181223,102073293,"""Nguyen Duc Thang""",2024-09-06,"""OFF""","""OFF""","""ThuFri""","""OFF""","""1900-0400"""


In [5]:
#[BKN]IEX🎯

Data_IEX = []
for file in glob.glob(Link_IEX):
    filename = os.path.basename(file)
    Read = pl.read_excel(file, infer_schema_length=None, sheet_name="Sheet1", engine="xlsx2csv")
    IEX = Read.select(pl.all(), pl.lit(filename).alias("Filename"))
    Data_IEX.append(IEX)
IEX = pl.concat(Data_IEX, how="diagonal")

#Edit Column
IEX = IEX.rename({'Personal ID': 'EID'}
        ).select(['Filename','ID Agent','First name Agent','Last name Agent','Suffix Agent','Unified user ID',
                  'External ID','EID','Address E-mail','Language E-mail','Date Seniority'])

# Export to Database
Export_IEX_DB = IEX.write_database("IEX_RAW", f"sqlite:///{Link_DB}", if_table_exists="replace")

#RUN
Export_IEX_DB
IEX.sample(n=3)

Filename,ID Agent,First name Agent,Last name Agent,Suffix Agent,Unified user ID,External ID,EID,Address E-mail,Language E-mail,Date Seniority
str,i64,str,str,str,str,str,i64,str,str,str
"""IEX.xlsx""",3003396,"""HUUNHATMINH""","""NGUYEN""",null,"""202403200249191025173290000000…","""huunhatminh.nguyen@concentrix.…",102517329,null,"""English (Philippines)""","""2024-03-18"""
"""IEX.xlsx""",3067250,"""TranPhan""","""Nguyen""",null,"""202305310238251022573340000000…","""tranphan.nguyen@concentrix.com""",102257334,null,"""English (Philippines)""","""2023-05-29"""
"""IEX.xlsx""",3100571,null,null,null,null,null,103208172,null,"""148""",null


In [6]:
#[BKN]Workplan🎯

Data_Workplan = []
for file in glob.glob(Link_Workplan):
    filename = os.path.basename(file)
    Read = pl.read_excel(file, infer_schema_length=None, sheet_name="Sheet1")
    Workplan = Read.select(pl.all(), pl.lit(filename).alias("Filename"))
    Data_Workplan.append(Workplan)
Workplan = pl.concat(Data_Workplan, how="diagonal_relaxed")

#Edit Column
Workplan = pl.DataFrame(Workplan).to_pandas() 
Workplan['Date'] = pd.to_datetime(Workplan['Date'], format='mixed').dt.date
Workplan['Start1'] = pd.to_datetime(Workplan['Start1'], format='mixed').dt.time
Workplan['End1'] = pd.to_datetime(Workplan['End1'], format='mixed').dt.time
Workplan['Start2'] = pd.to_datetime(Workplan['Start2'], format='mixed').dt.time
Workplan['End2'] = pd.to_datetime(Workplan['End2'], format='mixed').dt.time
Workplan = pl.from_pandas(Workplan)
# Workplan = Workplan.with_columns(pl.col('Start1','End1','Start2','End2').str.strptime(pl.Time, format='%H:%M'))
# Workplan = Workplan.with_columns(pl.col('Date').str.strptime(pl.Date, format='%m-%d-%y', strict=False))
Workplan = Workplan.with_columns(pl.col('ID').cast(pl.Int64))
threshold = t(15, 0, 0) # Định nghĩa ngưỡng thời gian (15:00:00)
Workplan = Workplan.with_columns( # Tính toán Date_end
    pl.when(pl.col("Start1").cast(pl.Time) >= threshold)
        .then(pl.col("Date").cast(pl.Date) + timedelta(days=1)) 
        .otherwise(pl.col("Date").cast(pl.Date)) 
        .alias("Date_end"))
Workplan = Workplan.with_columns( # Tạo DateTime_Start và DateTime_End trực tiếp từ chuỗi
    (pl.col("Date").dt.strftime("%Y-%m-%d") + " " + pl.col("Start1").cast(str)).str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("DateTime_Start"),
    (pl.col("Date_end").dt.strftime("%Y-%m-%d") + " " + pl.col("End1").cast(str)).str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("DateTime_End"))
Workplan = Workplan.filter(pl.col("Start2").is_not_null() & pl.col("End2").is_not_null()) # Filter for non-null values (similar to pandas .loc)
Workplan = Workplan.with_columns( # Create 'Date_Act_Start' column
    pl.when(
        (pl.col("Start1") >= t(15, 0, 0)) & (pl.col("Start2") <= t(15, 0, 0)))
    .then(pl.col("Date") + pl.duration(days=1))  # Directly add a duration to the Date column
    .otherwise(pl.col("Date"))
    .alias("Date_Act_Start"))
Workplan = Workplan.with_columns( # Create 'DateTime_Act_Start' column
    (pl.col("Date_Act_Start").dt.strftime("%Y-%m-%d") + " " + pl.col("Start2").cast(str))
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S")
    .alias("DateTime_Act_Start"))
Workplan = Workplan.with_columns( # Create 'Date_Act_End' column
    pl.when(
        (pl.col("Start1") >= t(15, 0, 0)) & (pl.col("End2") <= t(15, 0, 0)))
    .then(pl.col("Date") + pl.duration(days=1))  # Add duration directly to Date column
    .otherwise(pl.col("Date"))
    .alias("Date_Act_End"))
Workplan = Workplan.with_columns( # Create 'DateTime_Act_End' column
    (pl.col("Date_Act_End").dt.strftime("%Y-%m-%d") + " " + pl.col("End2").cast(str))
    .str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S")
    .alias("DateTime_Act_End"))
Workplan = Workplan.with_columns(( # Chuyển đổi kiểu dữ liệu và tính toán Dur (giờ)
    (pl.col("DateTime_End") - pl.col("DateTime_Start")).dt.total_seconds() / 3600).alias("Dur"))
Workplan = Workplan.with_columns(( # Chuyển đổi kiểu dữ liệu và tính toán Act_Dur (giờ)
    (pl.col("DateTime_Act_End") - pl.col("DateTime_Act_Start")).dt.total_seconds() / 3600).alias("Act_Dur"))
Workplan = Workplan.join(IEX,coalesce=True, left_on="ID", right_on="ID Agent", how="left"
                  ).select(['Filename','LOB','ID','EID','DateTime_Start','DateTime_End','Date','Date_end','Start1',
                            'End1','Dur','Schedule Act','DateTime_Act_Start','DateTime_Act_End','Date_Act_Start',
                            'Date_Act_End','Start2','End2','Act_Dur']
                  ).rename({'Date':'Date_Start','Start1':'Time_Start','End1':'Time_End','Schedule Act':'Action',
                            'Start2':'Time_Act_Start','End2':'Time_Act_End'})

# Export to Database
Export_Workplan_DB = Workplan.write_database("Workplan_RAW", f"sqlite:///{Link_DB}", if_table_exists="replace")

#RUN
Export_Workplan_DB
Workplan.sample(n=3)

Filename,LOB,ID,EID,DateTime_Start,DateTime_End,Date_Start,Date_end,Time_Start,Time_End,Dur,Action,DateTime_Act_Start,DateTime_Act_End,Date_Act_Start,Date_Act_End,Time_Act_Start,Time_Act_End,Act_Dur
str,str,i64,i64,datetime[μs],datetime[μs],date,date,time,time,f64,str,datetime[μs],datetime[μs],date,date,time,time,f64
"""20240901_20250302.xlsx""","""FR""",3079935,102369707,2024-12-07 13:00:00,2024-12-07 22:00:00,2024-12-07,2024-12-07,13:00:00,22:00:00,9.0,"""Email 1""",2024-12-07 15:00:00,2024-12-07 16:00:00,2024-12-07,2024-12-07,15:00:00,16:00:00,1.0
"""20231120_20240831.xlsx""","""EN""",3001380,102507451,2024-04-07 07:00:00,2024-04-07 16:00:00,2024-04-07,2024-04-07,07:00:00,16:00:00,9.0,"""Email 1""",2024-04-07 13:00:00,2024-04-07 14:10:00,2024-04-07,2024-04-07,13:00:00,14:10:00,1.166667
"""20250310_20250407.xlsx""","""EN""",3008652,102847116,2025-03-21 05:00:00,2025-03-21 14:00:00,2025-03-21,2025-03-21,05:00:00,14:00:00,9.0,"""Open Time""",2025-03-21 13:20:00,2025-03-21 14:00:00,2025-03-21,2025-03-21,13:20:00,14:00:00,0.666667


# For subsequent runs today, start from this section only 🤖

**Execute only the main code block to save time.**

In [7]:
# Library list🤖
import glob, warnings, polars as pl, datetime, sqlite3, time, os, zipfile, xml.dom.minidom
from datetime import datetime as dt, time as t, timedelta
import pandas as pd, numpy as np, sqlalchemy as sa, xlsxwriter
from sqlalchemy import create_engine
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from polars.exceptions import ColumnNotFoundError
import shutil

# -----------------------------------------------------------------------------------------------#
# Source collection
user_credential = os.path.join(os.environ['USERPROFILE'],r'Concentrix Corporation//CNXVN - WFM Team - Documents//')

# INPUT-----💾-----💾-----💾-----💾-----💾-----💾-----💾-----💾-----💾-----💾

# [BKN]AGENTS 0️⃣1️⃣
Link_AGENTS = os.path.join(user_credential,
                            r'DataBase//DataRaw//BKN//AGENTS//*.xlsx')
# [GLB]Ramco 0️⃣2️⃣
Link_Ramco = os.path.join(user_credential,
                            r'DataBase//DataRaw//GLOBAL//RAMCO//*.CSV')
# [BKN]Schedule 0️⃣3️⃣
Link_Schedule = os.path.join(user_credential,
                            r'DataBase//DataRaw//BKN//ROSTER//*.xlsx')
# [BKN]IEX 0️⃣4️⃣
Link_IEX = os.path.join(user_credential,
                        r'DataBase//DataRaw//BKN//IEX//*.xlsx')
# [BKN]Workplan 0️⃣5️⃣
Link_Workplan = os.path.join(user_credential,
                            r'DataBase//DataRaw//BKN//WORKPLAN//WORKPLAN QUERY//*.xlsx')
# [BKN]REALTIME_CUIC 0️⃣6️⃣
Link_REALTIME_CUIC = os.path.join(user_credential,
                            r'DataBase//DataRaw//BKN//REALTIME_VIEW//*.xlsx')

# OUTPUT-----📥-----📥-----📥-----📥-----📥-----📥-----📥-----📥-----📥-----📥

# [BKN]MONITOR_DF 📑
DF_MONITOR_DF = os.path.join(user_credential, 
                        r'DataBase//DataFrame//BKN//MONITOR_DF')

# -----------------------------------------------------------------------------------------------#
# MyMiscellaneous

# [DB]TonyMiscellaneous
Link_DB = os.path.join(os.environ['USERPROFILE'], r'Desktop//Bcom_DB.db')
conn = create_engine(f"sqlite:///{Link_DB}")

In [13]:
#[BKN]REALTIME_CUIC🎯
 
Data_REALTIME_CUIC = []
for file in glob.glob(Link_REALTIME_CUIC):
    filename = os.path.basename(file)
    Read = pl.read_excel(file, infer_schema_length=None, engine="calamine")
    REALTIME_CUIC = Read.select(pl.all(), pl.lit(filename).alias("Filename"))
    Data_REALTIME_CUIC.append(REALTIME_CUIC)
REALTIME_CUIC = pl.concat(Data_REALTIME_CUIC, how="diagonal")

#Edit Column
REALTIME_CUIC = REALTIME_CUIC.with_columns(
    pl.col("DateTime").str.to_datetime("%m/%d/%y %I:%M:%S %p").cast(pl.Datetime),
    pl.col('Extension').cast(pl.Int64, strict=False)).select(['Filename','DateTime','Agent','State','Reason','Direction','Duration','Extension'])

# Export to Database
Export_REALTIME_CUIC_DB = REALTIME_CUIC.write_database("REALTIME_CUIC_RAW", f"sqlite:///{Link_DB}", if_table_exists="replace")

# #RUN
Export_REALTIME_CUIC_DB
REALTIME_CUIC.sample(n=3)

Filename,DateTime,Agent,State,Reason,Direction,Duration,Extension
str,datetime[μs],str,str,str,str,f64,i64
"""00_RTA_View-Agent Team Real Ti…",2025-05-22 18:34:42,"""Ngo, Tracie""","""Not Ready""","""Picklist 2.0""","""Not Applicable""",0.000648,72786998
"""00_RTA_View-Agent Team Real Ti…",2025-05-22 17:03:02,"""Vo, Quinie""","""Not Ready""","""Picklist 2.0""","""Not Applicable""",0.06125,72786073
"""00_RTA_View-Agent Team Real Ti…",2025-05-22 17:38:42,"""Le, Nathan""","""Not Ready""","""Lunch""","""Not Applicable""",0.02912,72786823


In [14]:
#[BKN]MONITOR_DF🎡

query = """ 

WITH

---Setup CUIC REALTIME---🛖
REALTIME_CUIC AS (   
WITH
--Import Monitor, Masterroster 🤖
Monitor AS (Select Datetime([DateTime]) As [Code_Starttime],[Extension],[Agent],[State],[Reason],[Direction],[Duration],datetime(julianday([DateTime])+(Duration)) As [DateTime] From REALTIME_CUIC_RAW),
EmpID   As (Select [Employee_ID],[Extension Number],[CUIC Name],[Full name],[Role] From AGENTS_RAW),
ExtID   As (Select [Employee_ID],[Extension Number],[CUIC Name],[Full name],[Role] From AGENTS_RAW),
Roster  As (Select [Date],[Shift],[LOB],[team_leader] As [TeamLead],[Emp ID],[Name] From Schedule_RAW),
Leader  As (Select [Employee_ID],[TED Name] As [TL_Name],[cnx_email] As [TL_MAIL] From AGENTS_RAW)
--Run CUIC REALTIME 📟
Select Datetime(Monitor.[DateTime]) As [Actual Start], Monitor.[Code_Starttime],COALESCE(ExtID.[Employee_ID],EmpID.[Employee_ID]) As [Employee_ID], COALESCE(ExtID.[CUIC Name],EmpID.[CUIC Name]) As [CUIC Name], Monitor.[State] As [Current_State],
Case When Monitor.[Reason] = 'Picklist 2.0'      Then 'Email 1' 
     When Monitor.[Reason] = 'Lunch'             Then 'Lunch' 
     When Monitor.[Reason] = 'NONE'              Then 'NONE' 
     When Monitor.[Reason] = 'Outgoing Call'     Then 'Open Time' 
     When Monitor.[Reason] = 'Meeting'           Then 'Coaching 1:1' 
     When Monitor.[Reason] = 'CTI Failure'       Then 'CTI Failure' 
     When Monitor.[Reason] = 'Break'             Then 'Break Offline' 
     When Monitor.[Reason] = 'Not Working Yet'   Then 'Not Working Yet' 
     When Monitor.[Reason] = 'New Hire Training' Then 'New Hire Training' 
     When Monitor.[Reason] = 'Training'          Then 'Training Offline'
Else 'Other' End As [Current_Act],
Monitor.[Direction] As [Current_Direction], Monitor.[Duration]*24 As [Current_Duration], COALESCE(ExtID.[Full name],EmpID.[Full name]) As [Full name], COALESCE(ExtID.[Role],EmpID.[Role]) As [Role], Roster.[Shift], Roster.[LOB], Leader.[TL_Name], Leader.[TL_MAIL]
From Monitor
Left Join EmpID  On Monitor.[Agent] = EmpID.[CUIC Name]
Left Join ExtID  On Monitor.[Extension] = ExtID.[Extension Number]
Left Join Roster On Date(Monitor.[DateTime]) = Roster.[Date] And EmpID.[Employee_ID] = Roster.[Emp ID]
Left Join Leader On Roster.[TeamLead] = Leader.[Employee_ID]
),

---Setup WorkPlan---🛖
WorkPlan AS (    
--Import Workplan_RAW, Schedule_RAW, AGENTS_RAW 🤖
WITH
Workplan As (Select [Date_Act_Start] As [Date],[EID],[Action] As [WorkPlan_Act],[DateTime_Act_Start] As [Start],[DateTime_Act_End] As [End],[Act_Dur] As [WorkPlan_Dur] From Workplan_RAW),
Info As (
WITH
Sched As (Select [Date],[Original_Shift] As [Shift],[LOB],[team_leader] As [TeamLead],[Emp ID],[Name] From Schedule_RAW),
NameTL As (Select [Employee_ID],[TED Name] As [TL_Name],[cnx_email] As [TL_MAIL] From AGENTS_RAW)
Select Sched.[Date],Sched.[Shift],Sched.[LOB],NameTL.[TL_MAIL],NameTL.[TL_Name],Sched.[Emp ID],Sched.[Name] From Sched
Left join NameTL On Sched.[TeamLead] = NameTL.[Employee_ID] )
--Run Workplan 📟
Select 
Workplan.[Date],Info.[LOB],Info.[TL_MAIL],Info.[TL_Name] As [TeamLead],Workplan.[EID],Info.[Name],Info.[Shift],Workplan.[WorkPlan_Act],
Datetime(Workplan.[Start]) As [WorkPlan_Start],Datetime(Workplan.[End]) As [WorkPlan_End],Workplan.[WorkPlan_Dur] 
From Workplan
Left join Info On Workplan.[Date] = Info.[Date] And Workplan.[EID] = Info.[Emp ID]
Where strftime("%Y",Workplan.[Date]) >= '2024'
)

---Setup Monitor Screen---✨
Select 
COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])) As [Date], COALESCE(WorkPlan.[LOB],REALTIME_CUIC.[LOB]) As [LOB], COALESCE(WorkPlan.[TL_MAIL],REALTIME_CUIC.[TL_MAIL]) As [TL_MAIL], 
COALESCE(WorkPlan.[TeamLead],REALTIME_CUIC.[TL_Name]) As [TeamLead], COALESCE(WorkPlan.[EID],REALTIME_CUIC.[Employee_ID]) As [EID], COALESCE(WorkPlan.[Name],REALTIME_CUIC.[Full name]) As [Name], 
REALTIME_CUIC.[CUIC Name], REALTIME_CUIC.[Role], COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]) As [Shift], WorkPlan.[WorkPlan_Act], WorkPlan.[WorkPlan_Start], WorkPlan.[WorkPlan_End], 
WorkPlan.[WorkPlan_Dur], REALTIME_CUIC.[Actual Start], REALTIME_CUIC.[Code_Starttime], REALTIME_CUIC.[Current_Act], REALTIME_CUIC.[Current_Duration], REALTIME_CUIC.[Current_State], REALTIME_CUIC.[Current_Direction],
--Create Time in
Datetime(Date(CASE WHEN COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]) = '0000-0900' THEN Date(COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])), '-1 day') ELSE COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])) End),
TIME(SUBSTR(COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]),1,2)||':'||SUBSTR(COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]),3,2)||':'||'00', '-5 Minutes')) AS [LOGGIN],
--Create Time out
Datetime(Date(CASE WHEN COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]) IN ('1500-2400','1600-0100','1700-0200','1800-0300','1900-0400','2000-0500','2100-0600','2200-0700','2300-0800') THEN Date(COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])), '+1 day') ELSE COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])) END),
TIME(SUBSTR(COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]),6,2)||':'||SUBSTR(COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]),8,2)||':'||'00', '+5 Minutes')) AS [LOGGOUT],
/*    Setup scenario 📟                                                                                                                                                                          */
/* 1. OverLunch: [Current_Duration] > 1.083 And [Current_Act] = 'Lunch'                                                                                                                          */
(Case when REALTIME_CUIC.[Current_Duration] > 1.083 And REALTIME_CUIC.[Current_Act] = 'Lunch'           Then 'OverLunch,' Else '' End) ||
/* 2. OverBreak: [Current_Duration] > 0.283 And [Current_Act] = 'Break Offline'                                                                                                                  */
(Case when REALTIME_CUIC.[Current_Duration] > 0.283 And REALTIME_CUIC.[Current_Act] = 'Break Offline'   Then 'OverBreak,' Else '' End) ||
/* 3. OverACW: [Current_Duration] > 0.066 And [Current_Act] = 'Not Working Yet'                                                                                                                  */
(Case when REALTIME_CUIC.[Current_Duration] > 0.066 And REALTIME_CUIC.[Current_Act] = 'Not Working Yet' Then 'OverACW,'   Else '' End) ||
/* 4. Off-Day_Login: Shift in ('OFF','AL','CO','SL','VGH') And [Current_Act] IN ('Email 1', 'Coaching 1:1', 'Training Offline', 'Open Time') And CURRENT_TIME > 10:0:0                           */
(Case when COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]) In ('OFF','AL','CO','SL','HO','VGH') 
       And REALTIME_CUIC.[Current_Act] In ('Email 1', 'Coaching 1:1', 'Training Offline', 'Open Time') 
       And TIME(CURRENT_TIME,'+7 Hours') > TIME('10'||':'||'00'||':'||'00') Then 'Off-Day_Login,'   Else '' End) ||
/* 5. OutOfHoop: [WorkPlan_Act] is Null                                                                                                    */
(Case When WorkPlan.[WorkPlan_Act] is Null
/*And (REALTIME_CUIC.[Actual Start] Not Between 
--In
Datetime(Date(CASE WHEN COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]) = '0000-0900' THEN Date(COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])), '-1 day') ELSE COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])) End),
TIME(SUBSTR(COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]),1,2)||':'||SUBSTR(COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]),3,2)||':'||'00', '-5 Minutes'))
And
--Out
Datetime(Date(CASE WHEN COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]) IN ('1500-2400','1600-0100','1700-0200','1800-0300','1900-0400','2000-0500','2100-0600','2200-0700','2300-0800') THEN Date(COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])), '+1 day') ELSE COALESCE(WorkPlan.[Date],Date(REALTIME_CUIC.[Actual Start])) END),
TIME(SUBSTR(COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]),6,2)||':'||SUBSTR(COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]),8,2)||':'||'00', '+5 Minutes'))) */
Then 'OutOfHoop,' Else '' End) ||
/* 6. DropCode: [Current_Act] = 'CTI Failure'                                                                                                                                                     */
(Case when REALTIME_CUIC.[Current_Act] = 'CTI Failure' Then 'DropCode,'   Else '' End) ||
/* 7. WrongTrack: [Current_Act] <> [WorkPlan_Act] And [Workplan_Act] IN ('Email 1', 'Coaching 1:1', 'Training Offline', 'Open Time')
            Note: [Current_State] In ('Ready','Work Ready','Hold') And [Current_Act] = 'NONE' Consider as 'Open Time'
                  [LOB] = 'EN' And [WorkPlan_Act] = 'Open time' And [Current_Act] =  'Email 1' Consider as Valid                                                                                  */
(Case when REALTIME_CUIC.[Current_State] In ('Ready','Work Ready','Hold') And REALTIME_CUIC.[Current_Act] = 'NONE' And WorkPlan.[WorkPlan_Act] = 'Open Time' Then ''
      When COALESCE(WorkPlan.[LOB],REALTIME_CUIC.[LOB]) = 'EN' And WorkPlan.[WorkPlan_Act] = 'Open Time' And REALTIME_CUIC.[Current_Act] =  'Email 1' Then ''
      when REALTIME_CUIC.[Current_Act] <> WorkPlan.[WorkPlan_Act]  And WorkPlan.[WorkPlan_Act] In ('Email 1', 'Coaching 1:1', 'Training Offline', 'Open Time') Then 'WrongTrack' Else '' End) As [WFM_NOTE]
From WorkPlan
Full join REALTIME_CUIC On WorkPlan.[EID] = REALTIME_CUIC.[Employee_ID] And (REALTIME_CUIC.[Actual Start] BETWEEN WorkPlan.[WorkPlan_Start] AND WorkPlan.[WorkPlan_End])
Where REALTIME_CUIC.[Actual Start] Is Not Null And REALTIME_CUIC.[Role] = 'Agent'
Order by COALESCE(WorkPlan.[Shift],REALTIME_CUIC.[Shift]) Desc

    """
MONITOR_DF = pl.read_database(query=query, connection=conn.connect(), infer_schema_length=None)

#Edit Column
MONITOR_DF = MONITOR_DF.filter(pl.col("WFM_NOTE") != "")
MONITOR_DF = MONITOR_DF.select(['Date','LOB','TL_MAIL','TeamLead','EID','Name','CUIC Name','Shift','WorkPlan_Act','WorkPlan_Start','WorkPlan_End','Actual Start','Current_Act','Current_Duration','WFM_NOTE'])

# Export to CSV
os.chdir(DF_MONITOR_DF)
MONITOR_DF_CSV = MONITOR_DF.write_excel("BKN_MONITOR_DF.xlsx")

#RUN
MONITOR_DF_CSV
MONITOR_DF.sample(n=3)

Date,LOB,TL_MAIL,TeamLead,EID,Name,CUIC Name,Shift,WorkPlan_Act,WorkPlan_Start,WorkPlan_End,Actual Start,Current_Act,Current_Duration,WFM_NOTE
str,str,str,str,i64,str,str,str,str,str,str,str,str,f64,str
"""2025-05-22""","""IT""","""tungminh.tran@concentrix.com""","""Mason Tran""",102060823,"""Nguyen Cam Vinh""","""Nguyen, Eden""","""1400-2300""",null,null,null,"""2025-05-22 18:35:04""","""Lunch""",0.572778,"""OutOfHoop,"""
"""2025-05-22""",null,null,null,102512265,"""Lê Đặng Thanh Thảo""","""Le, Clara""",null,null,null,null,"""2025-05-22 18:14:00""","""Email 1""",0.007778,"""OutOfHoop,"""
"""2025-05-22""","""CS""","""vinhkhoi.le@concentrix.com""","""Karl Le""",102871042,"""Huynh Tuan Thong""","""Huynh, Danzel""","""1900-0400""",null,null,null,"""2025-05-22 18:35:42""","""Email 1""",1.094444,"""OutOfHoop,"""
